# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,slavgorod,52.9978,78.6449,-23.10,95,57,57,57,57
1,1,timbuktu,16.7735,-3.0074,21.46,18,1,1,1,1
2,2,berdigestyakh,62.0989,126.6992,-27.33,92,100,100,100,100
3,3,aldan,58.6031,125.3894,-32.00,97,100,100,100,100
4,4,blackmans bay,-43.0167,147.3167,16.52,61,65,65,65,65


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[
    (city_data_df['Max Temp'] > 20) &  # Filter for temperature greater than 20 degrees Celsius
    (city_data_df['Humidity'] < 50) &  # Filter for humidity less than 50%
    (city_data_df['Wind Speed'] < 10)  # Filter for wind speed less than 10 km/h
]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,timbuktu,16.7735,-3.0074,21.46,18,1,1,1,1
13,13,paoua,7.2427,16.4406,22.62,25,9,9,9,9
125,125,tazacorte,28.6290,-17.9293,21.75,43,0,0,0,0
153,153,tandil,-37.3217,-59.1332,25.38,49,2,2,2,2
176,176,faya-largeau,17.9257,19.1043,20.98,30,2,2,2,2
184,184,pergamino,-33.8900,-60.5736,28.99,24,0,0,0,0
216,216,seaside,36.6111,-121.8516,21.34,40,0,0,0,0
245,245,lompoc,34.6391,-120.4579,25.94,38,0,0,0,0
256,256,igbeti,8.7500,4.1333,25.36,18,6,6,6,6
283,283,guerrero negro,27.9769,-114.0611,23.60,23,0,0,0,0


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,timbuktu,1,16.7735,-3.0074,18,
13,paoua,9,7.2427,16.4406,25,
125,tazacorte,0,28.6290,-17.9293,43,
153,tandil,2,-37.3217,-59.1332,49,
176,faya-largeau,2,17.9257,19.1043,30,
184,pergamino,0,-33.8900,-60.5736,24,
216,seaside,0,36.6111,-121.8516,40,
245,lompoc,0,34.6391,-120.4579,38,
256,igbeti,6,8.7500,4.1333,18,
283,guerrero negro,0,27.9769,-114.0611,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 7000
params = {"categories": "accommodation.hotel",
    "apiKey":geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
timbuktu - nearest hotel: Hotel La Maison
paoua - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
tandil - nearest hotel: Viñas del Rosario Hotel Boutique
faya-largeau - nearest hotel: No hotel found
pergamino - nearest hotel: SIMOCA
seaside - nearest hotel: Embassy Suites
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
igbeti - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
tres arroyos - nearest hotel: Andrea Hotel
karratha - nearest hotel: Karratha International Hotel
carnarvon - nearest hotel: No hotel found
aldama - nearest hotel: Presas del rey
salinas - nearest hotel: The Traveler's Hotel
opuwo - nearest hotel: Ohakane Lodge
kelo - nearest hotel: No hotel found
dadiya - nearest hotel: No hotel found
al hilaliyya - nearest hotel: No hotel found
edd - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,timbuktu,1,16.7735,-3.0074,18,Hotel La Maison
13,paoua,9,7.2427,16.4406,25,No hotel found
125,tazacorte,0,28.6290,-17.9293,43,App Leyma
153,tandil,2,-37.3217,-59.1332,49,Viñas del Rosario Hotel Boutique
176,faya-largeau,2,17.9257,19.1043,30,No hotel found
184,pergamino,0,-33.8900,-60.5736,24,SIMOCA
216,seaside,0,36.6111,-121.8516,40,Embassy Suites
245,lompoc,0,34.6391,-120.4579,38,Embassy Suites by Hilton Lompoc Central Coast
256,igbeti,6,8.7500,4.1333,18,No hotel found
283,guerrero negro,0,27.9769,-114.0611,23,Plaza sal paraiso


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)